# Instrumental analysis requiring instruments in *cis*

## Libraries

In [1]:
library(readr)
library(dplyr, warn.conflicts = FALSE)
library(tidyr, warn.conflicts = FALSE)
library(purrr)
library(ggplot2)

## Reference panel

In [2]:
freq_1kg <- read_tsv("~/projects/DVA/Data/ReferenceData/1kg.afreq",
                     skip = 1,
                     col_names = c("chr", "position", "rsid", "nea_1kg_plus", "ea_1kg_plus", "eaf_1kg"),
                     col_types = "nncccn")

In [3]:
head(freq_1kg)

chr,position,rsid,nea_1kg_plus,ea_1kg_plus,eaf_1kg
<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1,11008,rs575272151,C,G,0.0884692
1,11012,rs544419019,C,G,0.0884692
1,13110,rs540538026,G,A,0.0566600
1,13116,rs62635286,T,G,0.1868790
1,13118,rs200579949,A,G,0.1868790
1,13273,rs531730856,G,C,0.1471170


## Outcome data - T2D

In [4]:
outcome_data <- read_tsv(
    pipe("sed 's/:/\t/g' ~/projects/DVA/Data/GWAS_sumstats/t2d_diagram.txt"),
    skip = 1,
    col_names = c("chr", "position", 
                  paste(c("ea", "nea", "beta", "se", "p", "n"), "t2d", sep = "_")),
    col_types = "nnccnnnn"
)
head(outcome_data)

chr,position,ea_t2d,nea_t2d,beta_t2d,se_t2d,p_t2d,n_t2d
<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
5,85928892,T,C,-0.0130,0.026,0.61,158186
11,107819621,A,C,-0.0710,0.170,0.67,124696
10,128341232,T,C,0.0140,0.012,0.24,158186
1,209652100,T,C,0.1500,0.130,0.25,131539
18,51112281,T,C,-0.0085,0.013,0.53,158184
4,55643311,T,C,-0.0060,0.016,0.71,158185


Harmonizing outcome data to reference panel:

In [5]:
outcome_data <- outcome_data %>%
    inner_join(freq_1kg, by = c("chr", "position")) %>%
    mutate(
        ea_1kg_minus = stringr::str_replace_all(ea_1kg_plus, c("A" = "t", "T" = "a", "C" = "g", "G" = "c")),
        ea_1kg_minus = toupper(ea_1kg_minus),
        nea_1kg_minus = stringr::str_replace_all(nea_1kg_plus, c("A" = "t", "T" = "a", "C" = "g", "G" = "c")),
        nea_1kg_minus = toupper(nea_1kg_minus),
        harmon = case_when(ea_t2d == ea_1kg_plus & nea_t2d == nea_1kg_plus ~ 1,
                           ea_t2d == ea_1kg_minus & nea_t2d == nea_1kg_minus ~ 1,
                           ea_t2d == nea_1kg_plus & nea_t2d == ea_1kg_plus ~ -1,
                           ea_t2d == nea_1kg_minus & nea_t2d == ea_1kg_minus ~ -1,
                           TRUE ~ 0)
    ) %>%
    filter(harmon != 0) %>%
    select(-c(harmon, ea_1kg_plus, ea_1kg_minus, nea_1kg_plus, nea_1kg_minus)) %>%
    # Ambiguous palindromic SNPs
    mutate(maf_1kg = ifelse(eaf_1kg < .5, eaf_1kg, 1 - eaf_1kg),
           palind_1 = ea_t2d %in% c("A", "T") & nea_t2d %in% c("A", "T"),
           palind_2 = ea_t2d %in% c("C", "G") & nea_t2d %in% c("C", "G"),
           palind_ambig = (palind_1 | palind_2) & maf_1kg > .4) %>%
    filter(!palind_ambig) %>%
    select(-starts_with("palind"), -eaf_1kg)
head(outcome_data)

chr,position,ea_t2d,nea_t2d,beta_t2d,se_t2d,p_t2d,n_t2d,rsid,maf_1kg
<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
5,85928892,T,C,-0.0130,0.026,0.61,158186,rs113534962,0.0626243
10,128341232,T,C,0.0140,0.012,0.24,158186,rs2366866,0.4592450
18,51112281,T,C,-0.0085,0.013,0.53,158184,rs62099898,0.2624250
4,55643311,T,C,-0.0060,0.016,0.71,158185,rs11725240,0.1640160
1,4429872,T,G,-0.0390,0.029,0.17,158184,rs72631092,0.0497018
5,58252088,A,C,-0.0080,0.016,0.62,158186,rs10069993,0.1590460


## Exposure data

### BMI

In [6]:
bmi <- read_tsv(
    "~/projects/DVA/Data/GWAS_sumstats/bmi.txt", 
    skip = 1,
    col_names = c("chr", "position", "rsid", 
                  paste(c("ea", "nea", "eaf", "beta", "se", "pval", "n"), "bmi", sep = "_")),
    col_types = "nncccnnnnn"
)
head(bmi)

chr,position,rsid,ea_bmi,nea_bmi,eaf_bmi,beta_bmi,se_bmi,pval_bmi,n_bmi
<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
7,92383888,rs10,A,C,0.06431,0.0013,0.0042,0.7500,598895
12,126890980,rs1000000,A,G,0.22190,0.0001,0.0021,0.9600,689928
4,21618674,rs10000010,T,C,0.50860,-0.0001,0.0016,0.9400,785319
4,1357325,rs10000012,C,G,0.86340,0.0047,0.0025,0.0570,692463
4,37225069,rs10000013,A,C,0.77080,-0.0061,0.0021,0.0033,687856
4,84778125,rs10000017,T,C,0.22840,0.0041,0.0021,0.0480,686123


### Data for each trait

Exposure data:

In [7]:
sigins <- read_tsv(
    "../data/sigins.tsv", 
    col_types = "nnnnnnccccnc"
)
head(sigins)

Warning message:
"One or more parsing issues, see `problems()` for details"


se,position,p,beta,chr,n,id,rsid,ea,nea,eaf,trait
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
0.0034143,111600265,1.65508e-10,-0.021820,1,344104,ukb-d-30730_irnt,rs2363605,A,G,0.12170,Gamma glutamyltransferase
0.0034060,111625213,1.54949e-10,-0.021801,1,344104,ukb-d-30730_irnt,rs2165145,T,C,0.12144,Gamma glutamyltransferase
0.0034022,111627231,1.95429e-10,-0.021656,1,344104,ukb-d-30730_irnt,rs12403848,T,C,0.12170,Gamma glutamyltransferase
0.0030641,111639354,3.49518e-08,-0.016898,1,344104,ukb-d-30730_irnt,rs325903,A,G,0.15575,Gamma glutamyltransferase
0.0033992,111643900,1.91280e-10,-0.021647,1,344104,ukb-d-30730_irnt,rs56296601,G,A,0.12180,Gamma glutamyltransferase
0.0030622,111645593,2.66048e-08,-0.017034,1,344104,ukb-d-30730_irnt,rs1282261,T,C,0.15587,Gamma glutamyltransferase


To select instruments:

- Gene IDs:

In [8]:
exposures_cismr <- tibble::tribble(
    ~trait_short, ~requireCisMR, ~id, ~ensembl_id,
    "SHBG", TRUE, "ukb-d-30830_irnt", "ENSG00000129214",
    "GGT", TRUE, "ukb-d-30730_irnt", paste(c("ENSG00000149435",
                                             "ENSG00000100121", 
                                             "ENSG00000274252", 
                                             "ENSG00000230712", 
                                             "ENSG00000276160", 
                                             "ENSG00000100031", 
                                             "ENSG00000133475", 
                                             "ENSG00000197421", 
                                             "ENSG00000280208", 
                                             "ENSG00000099998", 
                                             "ENSG00000167741", 
                                             "ENSG00000131067", 
                                             "ENSG00000236969"),
                                           collapse = ";"),
    "ALT", TRUE, "ukb-d-30620_irnt", paste(c("ENSG00000167701", "ENSG00000166123"), collapse = ";"),
    "ApoA1", TRUE, "ieu-b-107", "ENSG00000118137",
    "PTH", TRUE, "prot-a-2431", "ENSG00000152266",
    "CRP", TRUE, "ukb-d-30710_irnt", "ENSG00000132693"
)

In [9]:
exposures_cismr <- separate_rows(exposures_cismr, ensembl_id)

In [10]:
head(exposures_cismr)

trait_short,requireCisMR,id,ensembl_id
<chr>,<lgl>,<chr>,<chr>
SHBG,TRUE,ukb-d-30830_irnt,ENSG00000129214
GGT,TRUE,ukb-d-30730_irnt,ENSG00000149435
GGT,TRUE,ukb-d-30730_irnt,ENSG00000100121
GGT,TRUE,ukb-d-30730_irnt,ENSG00000274252
GGT,TRUE,ukb-d-30730_irnt,ENSG00000230712
GGT,TRUE,ukb-d-30730_irnt,ENSG00000276160


- Genomic regions in hg19 build:

In [11]:
gencode <- rtracklayer::readGFF("~/projects/DVA/Data/ReferenceData/gencode.v19.annotation.gtf")
head(gencode)

,seqid,source,type,start,end,score,strand,phase,gene_id,transcript_id,⋯,transcript_name,level,havana_gene,tag,havana_transcript,exon_number,exon_id,ont,protein_id,ccdsid
,<fct>,<fct>,<fct>,<int>,<int>,<dbl>,<chr>,<int>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,chr1,HAVANA,gene,11869,14412,NA,+,NA,ENSG00000223972.4,ENSG00000223972.4,⋯,DDX11L1,2,OTTHUMG00000000961.2,NA,NA,NA,NA,NA,NA,NA
2,chr1,HAVANA,transcript,11869,14409,NA,+,NA,ENSG00000223972.4,ENST00000456328.2,⋯,DDX11L1-002,2,OTTHUMG00000000961.2,basic,OTTHUMT00000362751.1,NA,NA,NA,NA,NA
3,chr1,HAVANA,exon,11869,12227,NA,+,NA,ENSG00000223972.4,ENST00000456328.2,⋯,DDX11L1-002,2,OTTHUMG00000000961.2,basic,OTTHUMT00000362751.1,1,ENSE00002234944.1,NA,NA,NA
4,chr1,HAVANA,exon,12613,12721,NA,+,NA,ENSG00000223972.4,ENST00000456328.2,⋯,DDX11L1-002,2,OTTHUMG00000000961.2,basic,OTTHUMT00000362751.1,2,ENSE00003582793.1,NA,NA,NA
5,chr1,HAVANA,exon,13221,14409,NA,+,NA,ENSG00000223972.4,ENST00000456328.2,⋯,DDX11L1-002,2,OTTHUMG00000000961.2,basic,OTTHUMT00000362751.1,3,ENSE00002312635.1,NA,NA,NA
6,chr1,ENSEMBL,transcript,11872,14412,NA,+,NA,ENSG00000223972.4,ENST00000515242.2,⋯,DDX11L1-201,3,OTTHUMG00000000961.2,NA,NA,NA,NA,NA,NA,NA


- Joining exposure IDs with genomic regions:

In [12]:
exposures <- gencode %>%
    filter(type == "gene") %>%
    transmute(chrom = as.numeric(gsub("chr", "", seqid)), start, end, ensembl_id = gsub("\\..*", "", gene_id)) %>%
    inner_join(exposures_cismr, by = "ensembl_id")
head(exposures)

Warning message in mask$eval_all_mutate(quo):
"NAs introduced by coercion"


,chrom,start,end,ensembl_id,trait_short,requireCisMR,id
,<dbl>,<int>,<int>,<chr>,<chr>,<lgl>,<chr>
1,1,159682079,159684379,ENSG00000132693,CRP,TRUE,ukb-d-30710_irnt
2,2,91963970,91969195,ENSG00000236969,GGT,TRUE,ukb-d-30730_irnt
3,8,145728356,145732557,ENSG00000167701,ALT,TRUE,ukb-d-30620_irnt
4,11,13513602,13517728,ENSG00000152266,PTH,TRUE,prot-a-2431
5,11,116706467,116708666,ENSG00000118137,ApoA1,TRUE,ieu-b-107
6,16,46918290,46965209,ENSG00000166123,ALT,TRUE,ukb-d-30620_irnt


- Now selecting instruments:

In [32]:
sigins_cis <- sigins %>%
    inner_join(exposures) %>%
    filter(chrom == chr) %>%
    mutate(within_gene = position > start & position < end,
           dist1 = abs(position - start),
           dist2 = abs(position - end)) %>%
    filter(within_gene | dist1 < 5e5 | dist2 < 5e5) %>%
    inner_join(bmi) %>%
    filter(pval_bmi < 5e-8)
head(sigins_cis)

Joining, by = "id"
Joining, by = c("position", "chr", "rsid")


se,position,p,beta,chr,n,id,rsid,ea,nea,⋯,within_gene,dist1,dist2,ea_bmi,nea_bmi,eaf_bmi,beta_bmi,se_bmi,pval_bmi,n_bmi
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,⋯,<lgl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.00411185,116747282,2.49977e-43,-0.0567477,11,NA,ieu-b-107,rs17120119,G,C,⋯,FALSE,40815,38616,C,G,0.93832,-0.0209,0.0036,9.8e-09,680967
0.00411260,116748221,1.80011e-43,-0.0568493,11,NA,ieu-b-107,rs12279433,T,G,⋯,FALSE,41754,39555,T,G,0.06168,0.0209,0.0036,8.4e-09,688386
0.00412167,116773318,4.60045e-43,-0.0567001,11,NA,ieu-b-107,rs17120132,A,T,⋯,FALSE,66851,64652,A,T,0.06086,0.0214,0.0036,3.7e-09,688842
0.00425460,116816394,1.69981e-36,-0.0536801,11,NA,ieu-b-107,rs10892050,A,C,⋯,FALSE,109927,107728,A,C,0.05718,0.0215,0.0037,8.0e-09,691844
0.00425965,116832718,2.19989e-36,-0.0536578,11,NA,ieu-b-107,rs10892051,A,C,⋯,FALSE,126251,124052,A,C,0.05741,0.0214,0.0037,9.5e-09,691844
0.00424763,116947180,1.90020e-36,-0.0535583,11,NA,ieu-b-107,rs11216265,C,T,⋯,FALSE,240713,238514,T,C,0.94685,-0.0219,0.0037,4.7e-09,691667


In [33]:
sigins_cis$trait_short %>% unique

[1] "ApoA1" "SHBG"

- Selecting instruments with strong effects on BMI and harmonizing:

In [26]:
validins_bmi <- sigins_cis %>%
    rename_with(~paste0(.x, "_exp"), c(ea, nea, eaf, beta, se, p)) %>%
    inner_join(outcome_data) %>%
    mutate(maf_bmi = ifelse(eaf_bmi < .5, eaf_bmi, 1 - eaf_bmi),
           new_ea = ifelse(beta_bmi > 0, ea_bmi, nea_bmi),
           new_nea = ifelse(beta_bmi > 0, nea_bmi, ea_bmi),
           eaf_bmi = ifelse(beta_bmi > 0, eaf_bmi, 1 - eaf_bmi),
           beta_bmi = abs(beta_bmi)) %>%
    select(-c(ea_bmi, nea_bmi)) %>%
    filter(abs(maf_bmi - maf_1kg) < .2) %>%
    ## Aligning to the BMI increasing allele
    mutate(
        ea_flip = toupper(stringr::str_replace_all(new_ea, c("A" = "t", "T" = "a", "C" = "g", "G" = "c"))),
        nea_flip = toupper(stringr::str_replace_all(new_nea, c("A" = "t", "T" = "a", "C" = "g", "G" = "c"))),
        harmon = case_when(ea_exp == new_ea & nea_exp == new_nea ~ 1,
                           ea_exp == new_nea & nea_exp == new_ea ~ -1,
                           ea_flip == new_ea & nea_flip == new_nea ~ 1,
                           ea_flip == new_nea & nea_flip == new_ea ~ -1,
                           TRUE ~ 0),
        beta_exp = beta_exp * harmon,
        beta_t2d = beta_t2d * harmon,
        ea = new_ea, nea = new_nea
    ) %>%
    ## Removing SNPs with allele mismatch
    filter(harmon != 0, abs(maf_bmi - maf_1kg) < .2) %>%
    select(-c(harmon, new_ea, new_nea, ea_flip, nea_flip))
head(validins_bmi)

Joining, by = c("position", "chr", "rsid")


se_exp,position,p_exp,beta_exp,chr,n,id,rsid,ea_exp,nea_exp,⋯,ea_t2d,nea_t2d,beta_t2d,se_t2d,p_t2d,n_t2d,maf_1kg,maf_bmi,ea,nea
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
0.00411185,116747282,2.49977e-43,-0.0567477,11,NA,ieu-b-107,rs17120119,G,C,⋯,C,G,0.031,0.025,0.23,152598,0.0646123,0.06168,G,C
0.00411260,116748221,1.80011e-43,-0.0568493,11,NA,ieu-b-107,rs12279433,T,G,⋯,T,G,-0.032,0.025,0.20,152598,0.0646123,0.06168,T,G
0.00412167,116773318,4.60045e-43,-0.0567001,11,NA,ieu-b-107,rs17120132,A,T,⋯,A,T,-0.035,0.025,0.15,158185,0.0646123,0.06086,A,T
0.00425460,116816394,1.69981e-36,-0.0536801,11,NA,ieu-b-107,rs10892050,A,C,⋯,A,C,-0.030,0.025,0.23,158185,0.0626243,0.05718,A,C
0.00425965,116832718,2.19989e-36,-0.0536578,11,NA,ieu-b-107,rs10892051,A,C,⋯,A,C,-0.030,0.025,0.23,158185,0.0626243,0.05741,A,C
0.00424763,116947180,1.90020e-36,-0.0535583,11,NA,ieu-b-107,rs11216265,C,T,⋯,T,C,0.037,0.027,0.16,158185,0.0626243,0.05315,C,T


In [ ]:
validins_bmi %>%
    

- Clumping to obtain independent SNPs:

In [15]:
clump_fx <- function(rsid, pval, 
                     plink = "/ludc/Tools/Software/Plink/v1.90b5.2/plink", 
                     bfile = "/ludc/Home/daniel_c/projects/DVA/Data/ReferenceData/1kg_ref/EUR",
                     clump_p = 5e-8, clump_kb = 500, clump_r2 = 0.01){
    fn <- tempfile()
    dat <- data.frame(SNP = rsid, P = pval)
    write_tsv(dat, fn)
    plink_cmd <- paste0(plink,
                        " --bfile ", bfile,
                        " --clump ", fn, 
                        " --clump-p1 ", clump_p, 
                        " --clump-r2 ", clump_r2,
                        " --clump-kb ", clump_kb, 
                        " --out ", fn)
    system(plink_cmd)
    clumped_snps <- paste("awk '{print $3}'", paste(fn, "clumped", sep = ".")) %>%
        pipe %>%
        readLines
    unlink(paste0(fn, "*"))
    return(clumped_snps)
}

In [30]:
cis_ins_dat <- validins_bmi %>%
    group_by(trait_short) %>%
    group_modify(~{
        snps_to_retain <- clump_fx(.x$rsid, .x$p_exp)
        clumped_dat <- filter(.x, rsid %in% snps_to_retain)
    }) %>%
    ungroup
cis_ins_dat

trait_short,se_exp,position,p_exp,beta_exp,chr,n,id,rsid,ea_exp,⋯,ea_t2d,nea_t2d,beta_t2d,se_t2d,p_t2d,n_t2d,maf_1kg,maf_bmi,ea,nea
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
ApoA1,0.00410692,116753552,6.89922e-44,-0.0570592,11,NA,ieu-b-107,rs12275565,G,⋯,A,G,0.0330,0.025,0.17,158185,0.0646123,0.06303,G,A
SHBG,0.00358100,7792465,3.84061e-26,0.0378770,17,312215,ukb-d-30830_irnt,rs2279620,C,⋯,C,G,0.0043,0.022,0.84,158182,0.1053680,0.10390,C,G


In [35]:
write_tsv(cis_ins_dat, "../data/cis_ins_dat.tsv")

In [36]:
cis_ins_dat %>%
    data.frame %>%
    print

  trait_short     se_exp  position       p_exp   beta_exp chr      n
1       ApoA1 0.00410692 116753552 6.89922e-44 -0.0570592  11     NA
2        SHBG 0.00358100   7792465 3.84061e-26  0.0378770  17 312215
                id       rsid ea_exp nea_exp  eaf_exp              trait
1        ieu-b-107 rs12275565      G       A 0.060807 apolipoprotein A-I
2 ukb-d-30830_irnt  rs2279620      C       G 0.113780               SHBG
      start       end      ensembl_id requireCisMR within_gene  dist1  dist2
1 116706467 116708666 ENSG00000118137         TRUE       FALSE  47085  44886
2   7517382   7536700 ENSG00000129214         TRUE       FALSE 275083 255765
  eaf_bmi beta_bmi se_bmi pval_bmi  n_bmi ea_t2d nea_t2d beta_t2d se_t2d p_t2d
1 0.06303   0.0206 0.0036  1.3e-08 688457      A       G   0.0330  0.025  0.17
2 0.10390   0.0170 0.0028  1.6e-09 674410      C       G   0.0043  0.022  0.84
   n_t2d   maf_1kg maf_bmi ea nea
1 158185 0.0646123 0.06303  G   A
2 158182 0.1053680 0.10390  C   G
